In [ ]:
import keras
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout


In [ ]:

# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Пути к наборам данных
train_dir = '/content/drive/MyDrive/Keras/train'
test_dir = '/content/drive/MyDrive/Keras/validation' 
# Размер изображений
img_width, img_height = 500, 500
# Размер мини-выборки
batch_size = 79
# Кол-во изображений для обучения
nb_train_samples = 1034
# Кол-во изображений для теста
nb_test_samples = 133

In [ ]:
# Создание генератора изображений
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1034 images belonging to 3 classes.
Found 133 images belonging to 3 classes.


In [ ]:
# Создание экземпляра модели сети MobileNet
Mobile_Net = MobileNet(
    input_shape=(img_width, img_height, 3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)
# input_shape : Необязательный кортеж формы
# альфа : контролирует ширину сети.
# depth_multiplier : множитель глубины для глубинной свертки.
# dropout : процент отсева.
# include_top : логическое значение, следует ли включать полностью подключенный уровень в верхнюю часть сети.
# weights : один из None(случайная инициализация), «imagenet» (предварительное обучение в ImageNet) или путь к загружаемому файлу весов. 
# input_tensor : Необязательный тензор Keras (т.е. вывод layers.Input()) для использования в качестве ввода изображения для модели.
# pooling : Дополнительный режим объединения для извлечения функций, когда include_top - False.
# classes : необязательное количество классов для классификации изображений, указывается только в том случае, если include_top установлено значение True и weights не указан
# classifier_activation : Функция активации для использования на «верхнем» слое.  

# Сверточную часть сети обучать не надо
Mobile_Net.trainable = False

Mobile_Net.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 500, 500, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 250, 250, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 250, 250, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 250, 250, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 250, 250, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 250, 250, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 250, 250, 32

In [ ]:
# Создание модели составной сети
model = Sequential()
# Добавляем сверточные слои
model.add(Mobile_Net)
# Преобразуем двумерный массив MobileNet в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Function (None, 15, 15, 1024)      3228864   
_________________________________________________________________
flatten_3 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               58982656  
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 62,211,777
Trainable params: 58,982,913
Non-trainable params: 3,228,864
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='SGD', 
              metrics=['accuracy'])


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=5,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
13/13 [==============================] - 191s 15s/step - loss: 0.6402 - accuracy: 0.6667 - val_loss: 0.6374 - val_accuracy: 0.6667
Epoch 2/5
13/13 [==============================] - 179s 14s/step - loss: 0.6407 - accuracy: 0.6667 - val_loss: 0.6376 - val_accuracy: 0.6667
Epoch 3/5
13/13 [==============================] - 180s 14s/step - loss: 0.6402 - accuracy: 0.6667 - val_loss: 0.6373 - val_accuracy: 0.6667
Epoch 4/5
13/13 [==============================] - 191s 15s/step - loss: 0.6401 - accuracy: 0.6667 - val_loss: 0.6372 - val_accuracy: 0.6667
Epoch 5/5
13/13 [==============================] - 185s 14s/step - loss: 0.6395 - accuracy: 0.6667 - val_loss: 0.6374 - val_accuracy: 0.6667


In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print('Точность работы на тестовых данных: %.2f%%' % (scores[1]*100))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1948: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Точность работы на тестовых данных: 66.67%
